# Emma Pan Neuro (Control + ND75KD) - pySCENIC pipeline (Embedded version)

**Author:** Vincent Gardeux

**Date Created:** 03/06/2024

# Libraries

In [1]:
# Fix OPENBLAS Warnings
import os
default_n_threads = 12
os.environ['OPENBLAS_NUM_THREADS'] = f"{default_n_threads}"
os.environ['MKL_NUM_THREADS'] = f"{default_n_threads}"
os.environ['OMP_NUM_THREADS'] = f"{default_n_threads}"

# import dependencies
import pandas as pd
import numpy as np
import h5py
import pickle
import pytz

from datetime import datetime
from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2
from distributed import Client, LocalCluster

from ctxcore.rnkdb import FeatherRankingDatabase as RankingDatabase
from pyscenic.binarization import binarize
from pyscenic.utils import modules_from_adjacencies
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

# Parameters

In [2]:
# [Input] Fixed gene annotation (for SCENIC feather file compatibility)
genome_data = pd.read_csv("/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/fixed_gene_annotation.tsv", sep = "\t", na_filter=False)

# [Input] Loom file to use
f_loom_path_scenic = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/Pan_neuro_both_reannotated_GFP_curated_reintegrated.loom"
# Open Loom file in reading mode
f = h5py.File(f_loom_path_scenic, 'r')
f_m = f["/matrix"][:,:]
f_gene_names = f["/row_attrs/Gene"].asstr()[:]
f_ensembl_names = f["/row_attrs/Accession"].asstr()[:]
f_cell_names = f["/col_attrs/CellID"].asstr()[:]
f.close()

# Update gene names using fixing table
ensembl_to_gene_dict = genome_data.set_index('Ensembl')['Name'].to_dict()
f_gene_names_updated = np.array([
    ensembl_to_gene_dict[ensembl] if ensembl in ensembl_to_gene_dict else gene
    for gene, ensembl in zip(f_gene_names, f_ensembl_names)
])

# Create expression matrix
ex_matrix = pd.DataFrame(f_m, columns = f_cell_names, index = f_gene_names_updated)
ex_matrix = ex_matrix.loc[f_gene_names_updated, :]

# [Input] Transcription factors list (SCENIC step 1: GRNBoost2)
#f_tfs = "/data/genome/drosophila_melanogaster/cistopic_flybase_r6.02/allTFs_dmel.txt" # From pySCENIC github
f_tfs = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/FBgg0000745_TF_Flybase.filtered.txt" # From Flybase
# Derive list of Transcription Factors(TF)
tf_names = load_tf_names(f_tfs)

# [Output] Adjacency matrix (SCENIC step 1: GRNBoost2)
adj_matrix = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_adj.csv"

# [Input] Ranking databases (SCENIC step 2-3: cisTarget)
f_db_names = ["/data/genome/drosophila_melanogaster/cistopic_flybase_r6.02/mc_v10_clust/dm6_v10_clust.genes_vs_motifs.rankings.feather"]
dbs = [RankingDatabase(fname=f_name, name=os.path.basename(f_name)) for f_name in f_db_names]

# [Input] Motif databases (SCENIC step 2-3: cisTarget)
f_motif_path = "/data/genome/drosophila_melanogaster/cistopic_flybase_r6.02/mc_v10_clust/motifs-v10-nr.flybase-m0.00001-o0.0.tbl"

# [Output] Regulons (SCENIC step 2-3: cisTarget)
f_motifs_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_motifs.tsv"
f_modules_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_modules.tsv"
f_regulons_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_regulons.tsv"
f_regulons_aucell_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_regulons_aucell.tsv"
f_regulons_binarized_aucell_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_regulons_aucell_binarized.tsv"
f_regulons_binarization_thresholds_aucell_path = "/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/filtered_tf_flybase/Pan_neuro_both_reannotated_GFP_curated_reintegrated_regulons_aucell_binarization_thresholds.tsv"

# Restrict matrix to feather genes
ranking_feather = pd.read_feather(f_db_names[0])
overlap_values = ex_matrix.index[pd.Series(ex_matrix.index).isin(ranking_feather.columns)].unique()
ex_matrix = ex_matrix.loc[overlap_values, :]

ex_matrix

,AAACCCAAGGTGATAT-1_ctrl,AAACCCACAAATAGCA-1_ctrl,AAACCCACAACAAAGT-1_ctrl,AAACCCACACTCATAG-1_ctrl,AAACCCACAGAGAGGG-1_ctrl,AAACCCACAGCCTATA-1_ctrl,AAACCCAGTACCTTCC-1_ctrl,AAACCCAGTACTGCCG-1_ctrl,AAACCCAGTGTTCGTA-1_ctrl,AAACCCAGTTCAGGTT-1_ctrl,...,TTTGTTGCAAGCACCC-1_ndkd,TTTGTTGCAGTTACCA-1_ndkd,TTTGTTGGTGAGATAT-1_ndkd,TTTGTTGGTGTCACAT-1_ndkd,TTTGTTGGTTAAGAAC-1_ndkd,TTTGTTGGTTTGGAGG-1_ndkd,TTTGTTGTCCGTTTCG-1_ndkd,TTTGTTGTCCTTCAGC-1_ndkd,TTTGTTGTCGAACGCC-1_ndkd,TTTGTTGTCGTGTTCC-1_ndkd
gfzf,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Osi24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
msps,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
CG6013,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
CR44091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Mitf,0.0,0.0,0.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,...,5.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0
Pur-alpha,3.0,6.0,3.0,1.0,0.0,15.0,11.0,3.0,0.0,0.0,...,2.0,3.0,5.0,1.0,1.0,2.0,1.0,6.0,1.0,0.0
gw,2.0,1.0,6.0,0.0,0.0,6.0,2.0,3.0,1.0,1.0,...,3.0,4.0,6.0,0.0,4.0,2.0,2.0,6.0,8.0,0.0
CR44027,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# SCENIC steps

## STEP 1: Gene regulatory network inference, and generation of co-expression modules

### 1.a. GRN inference using the GRNBoost2 algorithm

In the initial phase of the pySCENIC pipeline the single cell expression profiles are used to infer co-expression modules from.

Run GRNboost from arboreto to infer co-expression modules

The arboreto package is used for this phase of the pipeline.

*Output:* List of adjacencies between a TF and its targets.

Run GRNBoost2 algorithm

In [3]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

# Prepare the multithreading
cluster = LocalCluster(name='grn_call', dashboard_address=":12345", n_workers=default_n_threads, threads_per_worker=8)
client = Client(cluster)

# Here I run the function within the package (no CLI)
adjacencies = grnboost2(expression_data=ex_matrix.transpose(), tf_names=tf_names, seed=42, verbose=True, client_or_address=client)
            
# Shutting down cluster
client.close()
cluster.close()
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))
# Note: ~1h40 with n_workers=12, threads_per_worker=[not set] => default to 16 threads
# Note: ~2h10 with n_workers=12, threads_per_worker=8
# Note: 53mn with n_workers=12, threads_per_worker=8 => With reduced matrix

Start time: 18:13:30
preparing dask client
parsing input
creating dask graph
12 partitions
computing dask graph
not shutting down client, client was created externally
finished
End time: 18:59:23
Running time: 0:45:53.408936


Read in the adjacencies matrix

In [4]:
adjacencies.to_csv(adj_matrix, index=False, sep=',')
#adjacencies = pd.read_csv(adj_matrix, sep=',', na_filter=False) # If na_filter=True, the nan gene is detected as NaN
adjacencies

,TF,target,importance
395,bi,CR32773,8.560182e+02
478,CG9650,CR44357,4.424130e+02
573,salm,salr,4.024105e+02
491,br,Mur2B,3.896239e+02
474,Vsx2,Vsx1,3.817732e+02
...,...,...,...
300,rgr,Sox21a,2.229785e-17
125,REPTOR,CG9416,1.459898e-17
239,kni,CG30271,1.115953e-17
498,mamo,CCAP,5.907672e-20


## STEP 2-3: Regulon prediction aka cisTarget

*Output:* List of adjacencies between a TF and its targets.

### 2.a. Running regulon prediction using cisTarget

Here, we use the --mask_dropouts option, which affects how the correlation between TF and target genes is calculated during module creation. It is important to note that prior to pySCENIC v0.9.18, the default behavior was to mask dropouts, while in v0.9.18 and later, the correlation is performed using the entire set of cells (including those with zero expression). When using the modules_from_adjacencies function directly in python instead of via the command line, the rho_mask_dropouts option can be used to control this.

**Note:** Here I tried with and without the `rho_mask_dropouts=True` option. Without it, it generates 46 regulons. With it, 81 regulons. So I keep it.

**Update:** Now 99 regulons with fixed gene names, and `rho_mask_dropouts=True` option

**Update 2:** Now 108 regulons with fixed gene names, restricted to feather genes, and `rho_mask_dropouts=True` option. It contains dati and crc, but not repo anymore :(

**Update 3:** Now 764 regulons with filter_for_annotation=False ( no filtering on pruning )

**Update 4:** Now 579 regulons when using TF list from Flybase. 110 with basic pruning, and 289 with relaxed pruning (misses pros and sima).

In [37]:
modules = list(modules_from_adjacencies(adjacencies, ex_matrix.transpose(), rho_mask_dropouts=True, keep_only_activating=True)) # rho_mask_dropouts=True


2024-06-17 15:18:25,137 - pyscenic.utils - INFO - Calculating Pearson correlations.

2024-06-17 15:18:27,403 - pyscenic.utils - WARNING - Note on correlation calculation: the default behaviour for calculating the correlations has changed after pySCENIC verion 0.9.16. Previously, the default was to calculate the correlation between a TF and target gene using only cells with non-zero expression values (mask_dropouts=True). The current default is now to use all cells to match the behavior of the R verision of SCENIC. The original settings can be retained by setting 'rho_mask_dropouts=True' in the modules_from_adjacencies function, or '--mask_dropouts' from the CLI.
	Dropout masking is currently set to [True].

2024-06-17 15:19:11,231 - pyscenic.utils - INFO - Creating modules.


In [38]:
modules_df = pd.DataFrame(index = range(0, len(modules)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Context", "NES", "Markers"))
for j in range(0, len(modules)):
    # Setting values
    context = list(modules[j].context)
    modules_df["Regulon"].iloc[j] = modules[j].name
    modules_df["TF"].iloc[j] = modules[j].transcription_factor
    modules_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    modules_df["NbMarkers"].iloc[j] = len(set(modules[j].gene2weight))
    modules_df["Context"].iloc[j] = context[1]
    modules_df["NES"].iloc[j] = modules[j].score
    modules_df["Markers"].iloc[j] = ','.join(list(modules[j].gene2weight))

modules_df = modules_df.sort_values(by='NbMarkers', ascending=False)
modules_df.to_csv(f_modules_path, index=False, sep = "\t")
modules_df

,Regulon,TF,TFTargetGenesCorrelation,NbMarkers,Context,NES,Markers
3104,lola,lola,activating,7496,top50perTarget,0.0,"128up,14-3-3epsilon,14-3-3zeta,140up,18w,26-29..."
3003,bun,bun,activating,7319,top50perTarget,0.0,"128up,14-3-3epsilon,14-3-3zeta,18w,26-29-p,2mi..."
463,Regulon for lola,lola,weight>75.0%,7142,activating,0.0,"roX1,Cam,ps,14-3-3zeta,Atpalpha,sbb,Vha16-1,Rp..."
362,Regulon for bun,bun,weight>75.0%,7041,activating,0.0,"CG42613,zfh2,Rbfox1,ct,lola,RapGAP1,sbb,NFAT,S..."
2939,Pdp1,Pdp1,activating,6842,top50perTarget,0.0,"14-3-3epsilon,14-3-3zeta,140up,18w,26-29-p,2mi..."
...,...,...,...,...,...,...,...
1773,CG2889,CG2889,activating,20,top5perTarget,0.0,"CG10026,CG13164,CG14069,CG14327,CG17217,CG3106..."
1761,CG17612,CG17612,activating,20,top5perTarget,0.0,"CG15725,CG16716,CG16979,CG17068,CG3740,CG4159,..."
1730,CG10366,CG10366,activating,20,top5perTarget,0.0,"BCAS2,CG10347,CG11975,CG13151,CG14882,CG17118,..."
1789,CG4328,CG4328,activating,20,top5perTarget,0.0,"CG11997,CG13807,CG32150,CG33253,CG42564,CG4415..."


In [39]:
print(modules_df.TF.nunique(), "unique TF-modules were found ( out of",len(tf_names),"). Modules with less than 20 markers were filtered out.")

579 unique TF-modules were found ( out of 631 ). Modules with less than 20 markers were filtered out.


In [40]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

df = prune2df(dbs, modules, f_motif_path, num_workers=default_n_threads, weighted_recovery=True, rank_threshold = 1500, nes_threshold=1, motif_similarity_fdr=0.001, auc_threshold=0.05, filter_for_annotation=False)
    
end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))
# Note: 5mn37 with num_workers=48
# Note: 5mn29 with num_workers=12
# Note: 5mn33 with num_workers=10
# Note: 5mn37 with num_workers=8 ** optimal? 12mn with filter_for_annotation=False)
# Note: 8mn40 with num_workers=6
# Note: 9mn53 with num_workers=4
# Note: 30mn24 with num_workers=1

df.to_csv(f_motifs_path,sep = "\t")
#df

Start time: 17:24:41
End time: 18:01:29
Running time: 0:36:48.371595


In [41]:
print(len(set(df.index.get_level_values('TF').values)), "regulons were kept, after pruning")

579 regulons were kept, after pruning


In [42]:
# Look for main regulons
print("pros", "pros" in df.index.get_level_values('TF').values, sep="\t")
print("dati", "dati" in df.index.get_level_values('TF').values, sep="\t")
print("scro", "scro" in df.index.get_level_values('TF').values, sep="\t")
print("crc", "crc" in df.index.get_level_values('TF').values, sep="\t")
print("repo", "repo" in df.index.get_level_values('TF').values, sep="\t")
print("sima", "sima" in df.index.get_level_values('TF').values, sep="\t")

pros	True
dati	True
scro	True
crc	True
repo	True
sima	True


In [11]:
print("Size of Dataframe:", len(df))
drop_indexes = []
for j in range(0, len(df)):
    # Setting values
    if(len(df["Enrichment"]["TargetGenes"][j]) == 0): drop_indexes.append(df.index[j])
df_filtered = df["Enrichment"].drop(index=drop_indexes)
print("Size of Dataframe:", len(df_filtered))

Size of Dataframe: 453
Size of Dataframe: 453


These "modules" are then combined into regulons, by taking the top NES for each TF (for main Motif, and final score of regulon). All genes are bundled together.

In [12]:
# This dataframe can then be converted to regulons.
regulons = df2regulons(df_filtered)

Create regulons from a dataframe of enriched features.
Additional columns saved: []


In [13]:
regulon_df = pd.DataFrame(index = range(0, len(regulons)), columns = ("Regulon", "TF", "TFTargetGenesCorrelation", "NbMarkers", "Motif", "NES", "Markers"))
for j in range(0, len(regulons)):
    # Fixing order of set
    context = list(regulons[j].context)
    if(context[0].endswith(".png")):
        tmp = context[0]
        context[0] = context[1]
        context[1] = tmp
    # Setting values
    regulon_df["Regulon"].iloc[j] = regulons[j].name
    regulon_df["TF"].iloc[j] = regulons[j].transcription_factor
    regulon_df["TFTargetGenesCorrelation"].iloc[j] = context[0]
    regulon_df["NbMarkers"].iloc[j] = len(set(regulons[j].gene2weight))
    regulon_df["Motif"].iloc[j] = "https://resources.aertslab.org/cistarget/motif_collections/v10nr_clust_public/logos/" + context[1]
    regulon_df["NES"].iloc[j] = regulons[j].score
    regulon_df["Markers"].iloc[j] = ','.join(list(regulons[j].gene2weight))

regulon_df = regulon_df.sort_values(by='NbMarkers', ascending=False)
regulon_df.to_csv(f_regulons_path, index=False, sep = "\t")
regulon_df

,Regulon,TF,TFTargetGenesCorrelation,NbMarkers,Motif,NES,Markers
79,kay(+),kay,activating,1660,https://resources.aertslab.org/cistarget/motif...,5.187942,"RpS10b,Rph,rngo,PI31,kl-3,HERC2,Syx16,IntS2,Ma..."
1,Atf3(+),Atf3,activating,1308,https://resources.aertslab.org/cistarget/motif...,5.942495,"Surf4,pdgy,cbt,CG42813,CG43340,Galk,CG4729,Mit..."
36,Hnf4(+),Hnf4,activating,911,https://resources.aertslab.org/cistarget/motif...,2.17036,"Surf4,pdgy,GlyS,cbt,CG13579,CG5455,CG18812,FAS..."
101,srp(+),srp,activating,830,https://resources.aertslab.org/cistarget/motif...,6.264405,"GlyS,cbt,ush,CG13579,FASN1,CG13086,CG43340,Gal..."
76,grh(+),grh,activating,830,https://resources.aertslab.org/cistarget/motif...,7.410945,"Ac13E,slo,Dh31-R,shn,Atf3,cnc,wdb,CG8860,CG974..."
...,...,...,...,...,...,...,...
9,CG15011(+),CG15011,activating,4,https://resources.aertslab.org/cistarget/motif...,0.819731,"Osi8,dunk,CG4714,CG13003"
66,brk(+),brk,activating,4,https://resources.aertslab.org/cistarget/motif...,3.079079,"CG15117,CG42495,cher,raw"
108,vis(+),vis,activating,3,https://resources.aertslab.org/cistarget/motif...,0.162271,"CG11722,CG10194,CG11134"
8,CG14667(+),CG14667,activating,2,https://resources.aertslab.org/cistarget/motif...,3.226934,"CG33679,twe"


In [14]:
# Look for main regulons
print("pros", "pros" in regulon_df["TF"].values, sep="\t")
print("dati", "dati" in regulon_df["TF"].values, sep="\t")
print("scro", "scro" in regulon_df["TF"].values, sep="\t")
print("crc", "crc" in regulon_df["TF"].values, sep="\t")
print("repo", "repo" in regulon_df["TF"].values, sep="\t")
print("sima", "sima" in regulon_df["TF"].values, sep="\t")

pros	False
dati	False
scro	True
crc	True
repo	True
sima	False


## Phase III: Cellular regulon enrichment matrix (aka AUCell)

Characterize the different cells in a single-cell transcriptomics experiment by the enrichment of the regulons. Enrichment of a regulon is measures as AUC of the recovery curve of the genes that define this regulon.

In [15]:
auc_mtx = aucell(ex_matrix.transpose(), regulons, num_workers=default_n_threads)
auc_mtx.to_csv(f_regulons_aucell_path, sep = "\t")
auc_mtx

Regulon,Adf1(+),Atf3(+),Atf6(+),B-H1(+),B-H2(+),CG12391(+),CG12769(+),CG13894(+),CG14667(+),CG15011(+),...,sqz(+),srp(+),sv(+),tj(+),toe(+),twi(+),unpg(+),usp(+),vis(+),vvl(+)
Cell,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGGTGATAT-1_ctrl,0.00000,0.060894,0.000000,0.026088,0.00000,0.000000,0.133539,0.0,0.0,0.0,...,0.0,0.095986,0.026063,0.023969,0.0,0.000000,0.017791,0.176444,0.0,0.052597
AAACCCACAAATAGCA-1_ctrl,0.00000,0.028855,0.000000,0.051052,0.00000,0.000000,0.297820,0.0,0.0,0.0,...,0.0,0.038756,0.120817,0.320331,0.0,0.086766,0.071091,0.031373,0.0,0.079707
AAACCCACAACAAAGT-1_ctrl,0.00000,0.033274,0.572469,0.036353,0.00000,0.000000,0.262637,0.0,0.0,0.0,...,0.0,0.040248,0.081063,0.256901,0.0,0.066088,0.046544,0.039317,0.0,0.058137
AAACCCACACTCATAG-1_ctrl,0.03523,0.042875,0.264123,0.032304,0.00000,0.000000,0.278553,0.0,0.0,0.0,...,0.0,0.056384,0.060836,0.265665,0.0,0.017256,0.030919,0.024802,0.0,0.075927
AAACCCACAGAGAGGG-1_ctrl,0.00000,0.069539,0.000000,0.020045,0.00000,0.003564,0.047358,0.0,0.0,0.0,...,0.0,0.219836,0.071638,0.048890,0.0,0.015824,0.000000,0.131014,0.0,0.058505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTTGGAGG-1_ndkd,0.00000,0.180721,0.098223,0.109145,0.17448,0.000000,0.046611,0.0,0.0,0.0,...,0.0,0.131761,0.358830,0.087691,0.0,0.068428,0.001687,0.141249,0.0,0.105088
TTTGTTGTCCGTTTCG-1_ndkd,0.00000,0.040436,0.328142,0.047349,0.00000,0.000000,0.256037,0.0,0.0,0.0,...,0.0,0.061817,0.084317,0.232456,0.0,0.031809,0.011633,0.006242,0.0,0.076851
TTTGTTGTCCTTCAGC-1_ndkd,0.00000,0.048399,0.283184,0.055880,0.00000,0.000000,0.324815,0.0,0.0,0.0,...,0.0,0.046706,0.113251,0.311758,0.0,0.049231,0.013172,0.054190,0.0,0.048630


In [16]:
# Checkpoint to regenerate the object from the file
#auc_mtx = pd.read_csv("/home/gardeux/updepla_storage/users/gardeux/2023-04-19_snRNAseq_Emma/analysis/old/Pan_neuro_both_reannotated_GFP_curated_reintegrated_regulons_aucell.tsv", sep = "\t", index_col = "Cell")
auc_mtx = pd.read_csv(f_regulons_aucell_path, sep = "\t", index_col = "Cell")
auc_mtx.columns.name = "Regulon"
auc_mtx

Regulon,Adf1(+),Atf3(+),Atf6(+),B-H1(+),B-H2(+),CG12391(+),CG12769(+),CG13894(+),CG14667(+),CG15011(+),...,sqz(+),srp(+),sv(+),tj(+),toe(+),twi(+),unpg(+),usp(+),vis(+),vvl(+)
Cell,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGGTGATAT-1_ctrl,0.00000,0.060894,0.000000,0.026088,0.00000,0.000000,0.133539,0.0,0.0,0.0,...,0.0,0.095986,0.026063,0.023969,0.0,0.000000,0.017791,0.176444,0.0,0.052597
AAACCCACAAATAGCA-1_ctrl,0.00000,0.028855,0.000000,0.051052,0.00000,0.000000,0.297820,0.0,0.0,0.0,...,0.0,0.038756,0.120817,0.320331,0.0,0.086766,0.071091,0.031373,0.0,0.079707
AAACCCACAACAAAGT-1_ctrl,0.00000,0.033274,0.572469,0.036353,0.00000,0.000000,0.262637,0.0,0.0,0.0,...,0.0,0.040248,0.081063,0.256901,0.0,0.066088,0.046544,0.039317,0.0,0.058137
AAACCCACACTCATAG-1_ctrl,0.03523,0.042875,0.264123,0.032304,0.00000,0.000000,0.278553,0.0,0.0,0.0,...,0.0,0.056384,0.060836,0.265665,0.0,0.017256,0.030919,0.024802,0.0,0.075927
AAACCCACAGAGAGGG-1_ctrl,0.00000,0.069539,0.000000,0.020045,0.00000,0.003564,0.047358,0.0,0.0,0.0,...,0.0,0.219836,0.071638,0.048890,0.0,0.015824,0.000000,0.131014,0.0,0.058505
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTTGGAGG-1_ndkd,0.00000,0.180721,0.098223,0.109145,0.17448,0.000000,0.046611,0.0,0.0,0.0,...,0.0,0.131761,0.358830,0.087691,0.0,0.068428,0.001687,0.141249,0.0,0.105088
TTTGTTGTCCGTTTCG-1_ndkd,0.00000,0.040436,0.328142,0.047349,0.00000,0.000000,0.256037,0.0,0.0,0.0,...,0.0,0.061817,0.084317,0.232456,0.0,0.031809,0.011633,0.006242,0.0,0.076851
TTTGTTGTCCTTCAGC-1_ndkd,0.00000,0.048399,0.283184,0.055880,0.00000,0.000000,0.324815,0.0,0.0,0.0,...,0.0,0.046706,0.113251,0.311758,0.0,0.049231,0.013172,0.054190,0.0,0.048630


In [17]:
start = datetime.now(pytz.timezone('Europe/Paris'))
print("Start time:", start.strftime("%H:%M:%S"))

auc_mtx_bin = binarize(auc_mtx, seed = 42, num_workers=default_n_threads)

end = datetime.now(pytz.timezone('Europe/Paris'))
print("End time:", end.strftime("%H:%M:%S"))
print("Running time:", (end - start))

# Note: 23mn12 with num_workers=12

Start time: 19:08:14
End time: 19:10:38
Running time: 0:02:24.108463


In [18]:
binarization_thresholds = auc_mtx_bin[1]
binarization_thresholds.to_csv(f_regulons_binarization_thresholds_aucell_path, sep = "\t")
binarization_thresholds

Regulon
Adf1(+)    0.039416
Atf3(+)    0.118511
Atf6(+)    0.110401
B-H1(+)    0.097250
B-H2(+)    0.080305
             ...   
twi(+)     0.181587
unpg(+)    0.079221
usp(+)     0.127467
vis(+)     0.091568
vvl(+)     0.117457
Length: 110, dtype: float64

In [19]:
auc_mtx_bin = auc_mtx_bin[0]
auc_mtx_bin.to_csv(f_regulons_binarized_aucell_path, sep = "\t")
auc_mtx_bin

Regulon,Adf1(+),Atf3(+),Atf6(+),B-H1(+),B-H2(+),CG12391(+),CG12769(+),CG13894(+),CG14667(+),CG15011(+),...,sqz(+),srp(+),sv(+),tj(+),toe(+),twi(+),unpg(+),usp(+),vis(+),vvl(+)
Cell,,,,,,,,,,,,,,,,,,,,,
AAACCCAAGGTGATAT-1_ctrl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
AAACCCACAAATAGCA-1_ctrl,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
AAACCCACAACAAAGT-1_ctrl,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
AAACCCACACTCATAG-1_ctrl,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
AAACCCACAGAGAGGG-1_ctrl,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGTTTGGAGG-1_ndkd,0,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
TTTGTTGTCCGTTTCG-1_ndkd,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
TTTGTTGTCCTTCAGC-1_ndkd,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [20]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['scro(+)'] # 733 original (764 regulons, Aerts). 716 ok

14668

In [21]:
binarization_thresholds.loc['scro(+)'] # 0.3359530388214861 original (764 regulons, Aerts). 0.2968568380139414

0.2861698539340412

In [22]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['sima(+)'] # 1067 original (764 regulons, Aerts). 1212 ok

KeyError: 'sima(+)'

In [ ]:
binarization_thresholds.loc['sima(+)'] # 0.21179422980092633 original (764 regulons, Aerts). 0.18676990882955136

In [ ]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['crc(+)'] # 1136 original (764 regulons, Aerts). 1036 a bit too many?

In [ ]:
binarization_thresholds.loc['crc(+)'] # 0.21042246303356155 original (764 regulons, Aerts). 0.23696722508584317

In [ ]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['dati(+)'] # 11733 original (764 regulons, Aerts). 11631 ok

In [ ]:
binarization_thresholds.loc['dati(+)'] # 0.25299783006969817 original (764 regulons, Aerts). 0.2806370838941694

In [ ]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['pros(+)'] # 18482 original (764 regulons, Aerts). 18548 ok

In [ ]:
binarization_thresholds.loc['pros(+)'] # 0.11976533052607628 original (764 regulons, Aerts). 0.13361937192963938

In [ ]:
auc_mtx_bin.sum(numeric_only=True, axis=0).loc['repo(+)'] # 906 original (764 regulons, Aerts). 920 ok

In [ ]:
binarization_thresholds.loc['repo(+)'] # 0.10907608108082668 original (764 regulons, Aerts). 0.11348899982841607

In [ ]:
sum(auc_mtx["crc(+)"] > 0.238)

In [ ]:
sum(auc_mtx["crc(+)"] > binarization_thresholds.loc['crc(+)'])

In [ ]:
sum(auc_mtx_bin["crc(+)"])